# モデルを監視する

モデルをサービスとして運用環境にデプロイしたら、モデルを監視して使用状況を追跡し、処理する要求を調査します。Azure Application Insights を使用すると、モデル サービス エンドポイントのアクティビティを監視できます。

## ワークスペースに接続する

作業を開始するには、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## モデルをデプロイできるよう準備する

次に、デプロイするモデルが必要です。次のコードを実行します。

1. データセットを作成して登録します。
2. データセットを使用してモデルをトレーニングします。
3. モデルを登録します。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from azureml.core import Dataset

# Upload data files to the default datastore
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-data/',
                       overwrite=True,
                       show_progress=True)

#Create a tabular dataset from the path on the datastore
print('Creating dataset...')
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Register the tabular dataset
print('Registering dataset...')
try:
    data_set = data_set.register(workspace=ws, 
                               name='diabetes dataset',
                               description='diabetes data',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = data_set.to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
print('Registering model...')
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# Get the registered model
model = ws.models['diabetes_model']

print('Model trained and registered.')

## モデルを Web サービスとしてデプロイする

これで、登録済みモデルを Web サービスとしてデプロイする準備が整いました。

最初に、展開構成ファイル用フォルダーを作成します

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

现在需要一个入口脚本，此服务将使用它对新数据进行评分。

In [ ]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # Get a prediction from the model
    predictions = model.predict(np_data)
    
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

これで、サービスをデプロイできます (この場合は Azure Container Instances (ACI) として)。

> **注**: これには数分かかる場合があります。状態が**正常**と表示されるまで待ちます。

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# Configure the scoring environment
aci_service_env = Environment(name='aci_service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    aci_service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=aci_service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service-app-insights"
aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

## Application Insights を有効にする

次に、サービスの Application Insights を有効にする必要があります。

In [ ]:
# Enable AppInsights
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

## Web サービスを使用する

サービスをデプロイしたら、クライアント アプリケーションからサービスを使用できます。

まず、これらのアプリケーションが要求を送信する URL を決定します。

In [ ]:
endpoint = aci_service.scoring_uri
print(endpoint)

エンドポイント URI がわかったので、アプリケーションは HTTP 要求を行い、患者データを JSON (またはバイナリ) 形式で送信し、予測されたクラスを受け取ることができます。

> **ヒント**: サービス エンドポイントの準備が整っていないためエラーが発生した場合。数秒待ってからもう一度やり直してください!

In [ ]:
import requests
import json

# Create new data for inferencing
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

# Get the predictions
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

これで、サービス エンドポイントにについて記録されたデータを表示できます。

1. [Azure portal](https://portal.azure.com) で、Machine Learning ワークスペースを開きます。
2. **概要**ページで、関連付けられている **Application Insights** リソースのリンクをクリックします。
3. Application Insights ブレードで、**ログ**をクリックします。 

    > **注意**: 初めてログ分析を開く場合は、**はじめに**をクリックして、クエリ エディターを開く必要があります。クエリの書き込み方法を説明するヒントが表示されたら、それを閉じます。

4. 次のクエリをクエリ ウィンドウに貼り付けて、**実行**をクリックします
    ```
    traces
    |where  message == "STDOUT"
      and customDimensions.["Service Name"] == "diabetes-service-app-insights"
    |project timestamp, customDimensions.Content
    ```
5. 結果を表示します。ACI Web サービスは、テレメトリを Application Insights に送信するのに 5 分かかる場合があるため、最初は何も存在しない場合があります。数分待ってから、ログに記録されたデータと予測が表示されるまでクエリを再実行します。
6. ログに記録されたデータを確認したら、Application Insights クエリ ページを閉じます。

## サービスを削除する

不要になったサービスは削除する必要があります。

> **注**: サービスが使用中の場合は、すぐに削除できないことがあります。

In [ ]:
try:
    aci_service.delete()
    print('Service deleted.')
except Exception as ex:
    print(ex.message)

Application Insights を使用してデプロイされたサービスを監視する方法の詳細については、[Azure Machine Learning のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-enable-app-insights)を参照してください。